In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Importing the dataset from Kaggle to colab

In [ ]:
!pip install -q kaggle

In [ ]:
 !mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                  title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
teocalvo/teomewhy-loyalty-system                                     TeoMeWhy Loyalty System                           22MB  2024-06-14 19:10:56            128        352  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data       Online Sales Dataset - Popular Marketplace Data    7KB  2024-05-25 23:55:26           7609        141  1.0              
damirdizdarevic/uefa-euro-2024-players                               UEFA EURO 2024 - Players                          15KB  2024-06-08 18:07:29           1408         32  1.0              
informrohit1/smartphones-dataset                  

In [ ]:
! kaggle datasets download -d tongpython/cat-and-dog


Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
100% 217M/218M [00:00<00:00, 260MB/s]
100% 218M/218M [00:00<00:00, 238MB/s]


In [ ]:
! unzip -q cat-and-dog.zip


## Building the model


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(32,32,3)),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'),
    tf.keras.layers.Conv2D(filters =64, kernel_size =3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = 'valid')
])

In [ ]:
model.add(tf.keras.layers.Flatten())

In [ ]:
model.add(tf.keras.layers.Dropout(0.4))

In [ ]:
model.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
model.add(tf.keras.layers.Dense(units =1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 8, 8, 64)          0         
                                                                 
 dense (Dense)               (None, 8, 8, 128)         8

Fitting the CNN to images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
training_data_dir ='/content/training_set/training_set'
test_data_dir = '/content/test_set/test_set'

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = datagen.flow_from_directory(directory = training_data_dir, target_size = (32,32), classes=['dogs','cats'], class_mode='binary', batch_size =20)
test_set = datagen.flow_from_directory(directory = test_data_dir, target_size = (32,32), classes=['dogs','cats'], class_mode='binary', batch_size =20)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
len(training_set), len(test_set)

(401, 102)

In [ ]:
test_set.batch_size

20

In [ ]:
len(training_set)*20, len(test_set)*20

(8020, 2040)

In [ ]:
model.fit_generator(generator=training_set, steps_per_epoch=401, epochs= 20, validation_data = test_set, validation_steps =102)

Epoch 1/20


<ipython-input-34-a0fcdfe4f4e4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=training_set, steps_per_epoch=401, epochs= 20, validation_data = test_set, validation_steps =102)


401/401 [==============================] - 16s 34ms/step - loss: 0.6920 - accuracy: 0.5149 - val_loss: 0.6856 - val_accuracy: 0.5719
Epoch 2/20
401/401 [==============================] - 13s 33ms/step - loss: 0.6836 - accuracy: 0.5605 - val_loss: 0.6805 - val_accuracy: 0.5655
Epoch 3/20
401/401 [==============================] - 13s 32ms/step - loss: 0.6795 - accuracy: 0.5670 - val_loss: 0.6834 - val_accuracy: 0.5601
Epoch 4/20
401/401 [==============================] - 13s 33ms/step - loss: 0.6768 - accuracy: 0.5703 - val_loss: 0.6737 - val_accuracy: 0.5872
Epoch 5/20
401/401 [==============================] - 13s 32ms/step - loss: 0.6644 - accuracy: 0.5993 - val_loss: 0.6543 - val_accuracy: 0.6169
Epoch 6/20
401/401 [==============================] - 13s 33ms/step - loss: 0.6513 - accuracy: 0.6131 - val_loss: 0.6449 - val_accuracy: 0.6377
Epoch 7/20
401/401 [==============================] - 13s 33ms/step - loss: 0.6423 - accuracy: 0.6316 - val_loss: 0.6688 - val_accuracy: 0.5917
Epo

In [ ]:
def learning_curve(history, epoch):

    epochs_range = range(1 + len(model.history['accuracy']))
    plt.plot(epochs_range, history.history['accuracy'])
    plt.plot(epochs_range, history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(epochs_range, history.history['loss'])
    plt.plot(epochs_range, history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
new_var = learning_curve(history, 20)
new_var

NameError: name 'history' is not defined